In [5]:
#####Group Lassoによるマルチタスク学習#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [6]:
####データの生成####
##データの設定
g = 15   #グループ数
T = 30   #タスク数
n1 = np.random.poisson(np.random.gamma(7.5, 1/0.005, T))   
n2 = np.random.poisson(np.random.gamma(5.0, 1/0.005, T))   
N1 = np.sum(n1)
N2 = np.sum(n2)

In [7]:
##入力変数と応答変数を生成
#トピック分布の設定
k = 1000
topic = 30
theta = np.random.dirichlet(np.repeat(0.2, topic), T)
phi = np.random.dirichlet(np.repeat(0.2, k), topic)

#パラメータを生成
beta = np.vstack((np.random.normal(10.0, 1.5, T), np.random.multivariate_normal(np.repeat(0, T), np.diag(np.repeat(0.5, T)), k)))
beta[1:, ] = beta[1:, ] * np.random.binomial(1, 0.4, k)[:, np.newaxis]
beta[1:, ] = beta[1:, ] * np.random.binomial(1, 0.25, k*T).reshape(k, T)
Sigma = np.sqrt(1/np.random.gamma(10/2, 1/10, T))
betat = beta.copy(); Sigmat = Sigma.copy()

#タスクごとにデータを生成
x1_list = [i for i in range(T)]
x2_list = [i for i in range(T)]
y1_list = [i for i in range(T)]
y2_list = [i for i in range(T)]

for i in range(T):
    #トピックを生成
    z1 = np.dot(np.random.multinomial(1, theta[i, ], n1[i]), np.arange(topic))
    z2 = np.dot(np.random.multinomial(1, theta[i, ], n2[i]), np.arange(topic))

    #データを生成
    pt1 = np.random.poisson(np.random.gamma(12.5, 1/0.15, n1[i]))
    pt2 = np.random.poisson(np.random.gamma(12.5, 1/0.15, n2[i]))
    x1 = np.zeros((n1[i], k), dtype="int")
    x2 = np.zeros((n2[i], k), dtype="int")

    #学習データを生成
    for j in range(n1[i]):
        x1[j, ] = np.random.multinomial(pt1[j], phi[z1[j], ], 1).reshape(-1)
    x1_list[i] = np.hstack((np.repeat(1, n1[i])[:, np.newaxis], x1))
    y1_list[i] = np.dot(x1_list[i], beta[:, i]) + np.random.normal(0, Sigma[i], n1[i])

    #テストデータを生成
    for j in range(n2[i]):
        x2[j, ] = np.random.multinomial(pt2[j], phi[z2[j], ], 1).reshape(-1)
    x2_list[i] = np.hstack((np.repeat(1, n2[i])[:, np.newaxis], x2))
    y2_list[i] = np.dot(x2_list[i], beta[:, i]) + np.random.normal(0, Sigma[i], n2[i])

In [9]:
####Multitask Learning with Group Lassoを推定####
##データの定数を定義
#変数のターゲット
index_sort = np.array([np.sort(np.arange(k+1)), np.sort(np.arange(k+1))[::-1]])
index_target = np.zeros((k+1, k), dtype="int")
for j in range(k+1):
    index_target[j, ] = np.delete(np.arange(k+1), j)

#データの格納用配列
xx1_list = [i for i in range(k+1)]
xx2_list = [i for i in range(k+1)]
xy_list = [i for i in range(k+1)]

#変数ごとに定数を定義
for i in range(k+1):
    if i%100==0:
        print(i)
    xx1 = np.zeros((T, T))
    xx2 = np.zeros((T, k))
    xy = np.repeat(0, T)
    for j in range(T):
        x = x1_list[j]; y = y1_list[j]
        xx1[j, j] = np.dot(x[:, i].T, x[:, i])
        xx2[j, ] = np.dot(x[:, i].T, x[:, index_target[i, ]])
        xy[j] = np.dot(x[:, i].T, y)

    #リストに格納
    xx1_list[i] = xx1
    xx2_list[i] = xx2
    xy_list[i] = xy

0
100
200
300
400
500
600
700
800
900
1000


In [10]:
#変数をスパース変換
sparse_x1 = [i for i in range(T)]
sparse_y1 = [i for i in range(T)]
for i in range(T):
    sparse_x1[i] = sparse.csr_matrix(x1_list[i])
    sparse_y1[i] = sparse.csr_matrix(y1_list[i])

In [12]:
#初期値の設定
Lambda1 = 0.1
Lambda2 = Lambda1 * np.sqrt(T)
beta = np.zeros((k+1, T)); Sigma = np.repeat(0.0, T)
for j in range(T):
    x = x1_list[j]; y = y1_list[j]
    xy = np.dot(x.T, y)
    inv_xxv = np.linalg.inv(np.dot(x.T, x) + np.diag(np.repeat(2.5, k+1)))
    beta[:, j] = np.dot(inv_xxv, xy)
    Sigma[j] = np.sqrt(np.sum(np.power(y - np.dot(x, beta[:, j]), 2)) / n1[j])
sparse_beta = sparse.csr_matrix(beta) 

In [ ]:
##座標降下法による推定
for rp in range(index_sort.shape[0]):
    for i in index_sort[rp]:
        #データを抽出
        index = index_target[i, ]
        beta_target = sparse_beta[index, ]

        #回帰パラメータを更新
        gamma = np.repeat(0.0, T)
        for j in range(T):
            x11 = sparse_x1[j]; y11 = sparse_y1[j]
            x12 = x1_list[j]; y12 = y1_list[j]
            er_y = np.array((y11.T - np.dot(x11[:, index], beta_target[:, j])).todense()).reshape(-1)
            gamma[j] = np.dot(x12[:, i], er_y) / n1[j]

        #軟閾値作用素により回帰パラメータを縮小
        z = np.sign(gamma)
        sparse_beta[i, ] = (gamma - z*Lambda2) * np.array(np.abs(gamma - z*Lambda2) > Lambda2, dtype="int")

    #平均二乗誤差和を更新
    beta = np.array(sparse_beta.todense())
    LL = np.repeat(0.0, T)
    for i in range(T):
        x = x1_list[i]; y = y1_list[i]
        mu = np.dot(x, beta[:, i])
        LL[i] = np.sum(np.power(y - mu, 2)) / n2[i]
    print(np.sum(LL) + Lambda2 * np.sum(np.power(np.diag(np.dot(beta.T, beta)), 1/2)))

In [ ]:
#初期値の設定
Lambda1 = 0.1
Lambda2 = Lambda1 * np.sqrt(T)
rho = 0.25
beta[0, ] = np.array([np.mean(y1_list[j]) for j in range(T)])
beta[1:, ] = np.random.multivariate_normal(np.repeat(0, T), np.diag(np.repeat(0.25, T)), k)
gamma = beta + np.random.multivariate_normal(np.repeat(0, T), np.diag(np.repeat(0.1, T)), k+1)
u = beta + np.random.multivariate_normal(np.repeat(0, T), np.diag(np.repeat(0.1, T)), k+1)
sparse_beta = sparse.csr_matrix(beta)
Sigma = np.repeat(1, T)

In [ ]:
##交互方向乗数法による推定
for rp in range(100):
    #betaを更新
    for i in range(T):
        xx1 = xx1_list[i]; xx2 = xx2_list[i]; xy = xy_list[i]
        inv_xxv = np.linalg.inv(xx1 + rho*np.diag(n1))
        beta[i, ] = np.dot(inv_xxv, xy - np.diag(np.dot(xx2, beta[index_target[i, ]])) + n1*rho*(gamma[i, ] - (1/rho)*u[i, ]))
        
    #軟閾値作用素によりgammaを更新
    gamma = beta + u/rho
    z = np.sign(gamma)
    gamma = ((gamma - z*Lambda2) * np.array(np.abs(gamma - z*Lambda2) > Lambda2, dtype="int"))

    #uを更新
    u = u + rho*(beta - gamma)

    #平均二乗誤差和を更新
    LL = np.repeat(0.0, T)
    for i in range(T):
        x = x1_list[i]; y = y1_list[i]
        mu = np.dot(x, gamma[:, i])
        LL[i] = np.sum(np.power(y - mu, 2)) / n1[i]
    print(np.sum(LL) + Lambda2 * np.sum(np.power(np.diag(np.dot(beta.T, beta)), 1/2)))